In [0]:
%run ../Utilities/ConfigUtilities

In [0]:
# Set up the environment using a function in ConfigUtilties.
set_spark_config()

In [0]:
clean_out_df = spark.read.table(MDM_CLEANED_TABLE)
print(clean_out_df.count())
display(clean_out_df)

In [0]:
from delta.tables import DeltaTable

# Roll back changes.  Find the target.  
clean_table = DeltaTable.forPath(spark, MDM_CLEANED_PATH)
history_df = clean_table.history()

display(history_df)


In [0]:
clean_table.restoreToVersion(126)

history_df = clean_table.history()

display(history_df)

In [0]:
# Re-read the data to check the changes.
clean_out_df = spark.read.table(MDM_CLEANED_TABLE)
print(clean_out_df.count())


In [0]:
 # Check for duplicates at this point,  
duplicates_out_df = clean_out_df.groupBy("MeterNumber", "UnitOfMeasure", "FlowDirection", "Channel", "StartDateTime", "EndDateTime")\
                  .count().filter("count > 1")

dup_count = duplicates_out_df.count()

if dup_count > 0: 
  print(dup_count)
  display(duplicates_out_df)

  clean_out_dup_df = duplicates_out_df.join(clean_out_df, ["MeterNumber", "UnitOfMeasure", "FlowDirection", "Channel", "StartDateTime", "EndDateTime"], "left")
  print(clean_out_dup_df.count())
  display(clean_out_dup_df.orderBy(["MeterNumber", "StartDateTime"], ascending=True))


  #raise Exception("Duplicates found in the clean table.  Please investigate.")

else:
  print("No duplicates found after the upsert.")

In [0]:
# Get the change history
starting_version = 100

clean_changes_all_df = spark.read \
        .option("readChangeFeed", "true") \
        .option("startingVersion", starting_version) \
        .table(MDM_CLEANED_TABLE)

#        .option("startingTimestamp", last_change) \


In [0]:
from pyspark.sql.functions import col

# Filter down to a specific meter and timeframe.
sample_df = clean_changes_all_df.filter((col("MeterNumber") == 18778314) & (col("StartDateTime") == "2025-01-25T11:45:00.000+00:00") )

display(sample_df)